주차 수요 예측 AI 경진대회 https://dacon.io/competitions/official/235745/overview/description

목표 : 🏠 유형별 임대주택 설계 시 단지 내 적정 🅿️ 주차 수요를 예측 (차량등록수)

<br>

In [25]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc

import seaborn as sns

# # Mac 유저
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# Windows 유저
# plt.rcParams['font.family'] = 'Malgun Gothic'

import missingno as msno

<br>

# 데이터 분석 및 전처리

## 데이터셋

In [4]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
test = pd.read_csv('data/test.csv', encoding='utf-8')

<br>

## 데이터 분석

In [5]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [7]:
train.groupby(['단지코드']).nunique(dropna=False).sum(axis=0)

총세대수                             423
임대건물구분                           456
지역                               423
공급유형                             488
전용면적                            1898
전용면적별세대수                        2230
공가수                              423
자격유형                             510
임대보증금                           1277
임대료                             1289
도보 10분거리 내 지하철역 수(환승노선 수 반영)     423
도보 10분거리 내 버스정류장 수               423
단지내주차면수                          423
등록차량수                            423
dtype: int64

<br>

## 데이터 전처리

### 단지코드 중복 해결

<span style='color:red;'/> <b> 하나의 단지코드에 대해 둘 이상의 항목 존재 </b>

- 임대건물구분
- 공급유형
- 자격유형
- 전용면적
- 전용면적별세대수
- 임대보증금
- 임대료

#### 단지코드에 따라 단일값을 가지는 컬럼

In [8]:
unique_cols = ['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수']
train_new = train.set_index('단지코드')[unique_cols].drop_duplicates()
train_new.head()

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,
C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0


In [10]:
len(train_new)

423

<br>

#### 임대건물구분

In [11]:
# 하나의 단지코드에 여러 값이 있는 카테고리열(임대건물구분) 피벗테이블 생성
def reshape_building_cat(df, cat_col, val_col):
    return df.drop_duplicates(['단지코드', cat_col]).assign(counter=1).pivot(index='단지코드', columns=cat_col, values=val_col).fillna(0)

In [12]:
categorized_df1 = reshape_building_cat(train, '임대건물구분', '임대건물구분').replace(['상가', '아파트'], 1)
categorized_df1.head()

임대건물구분,상가,아파트
단지코드,,
C1000,0,1
C1004,1,1
C1005,0,1
C1013,0,1
C1014,0,1


In [13]:
def add_building_column(df, df_new):
    categorized_df1 = reshape_building_cat(df, '임대건물구분', '임대건물구분').replace(['상가', '아파트'], 1)
    df_new = df_new.join(categorized_df1)
    df_new.rename(columns={'상가': '임대건물구분_상가', '아파트': '임대건물구분_아파트'}, inplace=True)
    return df_new

In [14]:
train_new = add_building_column(train, train_new)
train_new.head()

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,임대건물구분_상가,임대건물구분_아파트
단지코드,,,,,,,,,
C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0,0,1
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0,0,1
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0,0,1
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0,0,1
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0,0,1


<br>

#### 공급유형, 자격유형

In [15]:
# 공급유형 & 자격유형 관계
pd.crosstab(train['공급유형'], train['자격유형'] , margins = True).style.background_gradient(cmap = 'coolwarm')

자격유형,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,All
공급유형,,,,,,,,,,,,,,,,
공공분양,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,7
공공임대(10년),205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,205
공공임대(50년),31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31
공공임대(5년),3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
공공임대(분납),12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
국민임대,1539,21,0,0,34,0,9,155,0,0,0,0,0,0,0,1758
영구임대,2,0,95,0,3,3,0,0,49,0,0,0,0,0,0,152
임대상가,0,0,0,562,0,0,0,0,0,0,0,0,0,0,0,562
장기전세,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9


In [16]:
# 공공임대(10년), 공공임대(50년), 공공임대(5년), 공공임대(분납) -> 공공임대
def merge_type_cat(type):
    if type.startswith('A_공공임대'):
        return 'A_공공임대'
    else:
        return type

In [20]:
# 자격_공급 유형 컬럼 추가
def add_new_type(df):
    df['자격_공급'] = df['자격유형'] + '_' + df['공급유형']
    return df

In [23]:
train = add_new_type(train)
type_lists = list(train['자격_공급'].unique()) # 전역(train/test)

In [22]:
# 중복 제거된 데이터프레임에 자격_공급 카테고리 컬럼 추가
def add_type_column(df, df_new):

    # df_new - 자격_공급 유형 새로운 열 추가
    for type in type_lists:
        df_new[type]=0

    for code in df_new.index:
        for type in type_lists:
            df_new[type][code] = len(df[(df['자격_공급']==type) & (df['단지코드']==code)])
            
    return df_new

In [24]:
train_new = add_type_column(train, train_new)
train_new.head()

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,임대건물구분_상가,임대건물구분_아파트,A_국민임대,...,I_영구임대,D_공공분양,A_영구임대,J_행복주택,K_행복주택,L_행복주택,A_공공임대(5년),M_행복주택,N_행복주택,O_행복주택
단지코드,,,,,,,,,,,,,,,,,,,,,
C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0,0,1,8,...,0,0,0,0,0,0,0,0,0,0
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0,0,1,7,...,0,0,0,0,0,0,0,0,0,0
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0,0,1,11,...,0,0,0,0,0,0,0,0,0,0
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0,0,1,4,...,0,0,0,0,0,0,0,0,0,0


<br>

#### 전용면적

In [ ]:
def

<br>

#### 전용면적별 세대수

<br>

#### 임대료

<br>

#### 임대보증금

<br>

### 결측치 제거

In [ ]:
train_copy = train.copy()
train = train_new

In [ ]:
msno.matrix(train, figsize=(15,10))

In [ ]:
msno.heatmap(train)

In [ ]:
train.isnull().sum()

#### 도보 10분거리 내 지하철역 수(환승노선 수 반영)

In [ ]:
train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0, inplace=True)
train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull().sum()

<br>

### 추가자료

#### 2020년_12월_자동차_등록자료_통계.xlsx

https://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1257

지역별 인구수와 자동차 등록 통계자료 이용하여 지역 점수 부과

In [ ]:
# 추가자료 - 2020년 12월 자동차 등록 통계자료
car = pd.read_excel('data/자동차_등록자료_통계.xlsx', index_col='구분')
car = car.iloc[:,:16]
car.head()

In [ ]:
# dacon 제공자료 - 지역별 & 성별 & 연령별 인구 데이터
age = pd.read_csv('data/age_gender_info.csv', encoding='utf-8',index_col = '지역')
age.head()

In [ ]:
temp_lst = set()

for x in car.columns:
    temp_lst.add(x)
    
temp_lst = list(temp_lst)
temp_lst.sort()

np.score = np.array(age.sort_index()) * np.array(car.transpose().sort_index())
df_score = pd.DataFrame(np.score)
df_score['지역'] = temp_lst
df_score['지역점수'] = df_score.transpose().iloc[:22,:].sum().transpose()

In [ ]:
df_score[['지역', '지역점수', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].head()

In [ ]:
df_score = df_score[['지역', '지역점수']]

train = pd.merge(train, df_score, how='left', on='지역')
test = pd.merge(test, df_score, how='left', on='지역')

In [ ]:
train.head()

<br>

## 테스트 데이터 전처리

In [29]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


### 단지코드 중복제거

In [ ]:
test_new = test.set_index('단지코드')[unique_cols].drop_duplicates()


<br>

# 머신러닝